notes:
- improve embedding : fast eigen decomp using power method or qr projection
    then choose $k$, then do eigsh decomp
- more efficient clustering, read more on fiedler/perron characteristics
- improve $eq_h$ decomposition for tailored to maximum kelmans res

In [7]:
import networkx as nx
import numpy as np
import scipy.sparse as sps
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans
import example_graphs as exgr
import spectral_embedding as spe

In [8]:
def power_method(A, tol=1e-6, max_iter=1000, x0=None):
    n = A.shape[0]
    if x0 is None or np.allclose(x0, 0):
        x = np.ones(n)
    else:
        x = x0
    x = x / np.linalg.norm(x)
    lam_old = 0
    for i in range(max_iter):
        x = A.dot(x)
        norm = np.linalg.norm(x)
        if norm == 0:
            return 0, x, i
        x = x / norm
        lam = np.dot(x, A.dot(x))
        if abs(lam - lam_old) < tol:
            return lam, x, i+1
        lam_old = lam
    return lam, x, max_iter

In [2]:
def spectral_radius_power(G, max_iter=1000, tol=1e-6):
    A = nx.adjacency_matrix(G)
    n = A.shape[0]
    x = np.random.rand(n)
    x /= np.linalg.norm(x)
    for _ in range(max_iter):
        x_new = A @ x
        x_new_norm = np.linalg.norm(x_new)
        x_new /= x_new_norm

        if np.linalg.norm(x - x_new) < tol:
            break
        x = x_new
    return x_new_norm

In [ ]:
n = 25
G = exgr.test_graph(n)

A = spe.computeA(G)
lam,x,_ =power_method(A)


7.205616713572342
[-4.66726249  7.20561722]


In [21]:
def QR_orthogonalization(A, k, p=10, q=2,):
    n = A.shape[0]
    O = np.random(n,k+p)
    Y = A.dot(O) if hasattr(A, "dot") else np.dot(A, O)
    for _ in range(q):
        Y = A.dot(Y) if hasattr(A, "dot") else np.dot(A,O)

    Q, _ = np.linalg.qr(Y, mode='reduced')
    B = Q.T.dot(A.dot(Q)) if hasattr(A,"dot") else np.dot(Q.T, np.dot(A,Q))
    evals, evecs_B = np.linalg.eigh(B)
    idx = np.argsort(evals)
    evals = evals[idx]
    evecs_B = evecs_B[:,idx]
    evecs = Q.dot(evecs_B[:,:k])
    evals = evals[:k]
    return evals, evecs